In [ ]:
# This is actually dremovd@github code for calculating coco metric. 

In [1]:
import sys
import pandas as pd

In [2]:
import os

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

Create keras model and load weights

In [3]:
from model import get_testing_model
model = get_testing_model()

Using TensorFlow backend.


In [4]:
from coco_metric import per_image_scores, validation

In [6]:
training_dir = './training/'
trained_models = [
    'weights'
    #'weights-cpp-lr',
    #'weights-python-last',
]
optimal_epoch_loss = 'val_weight_stage6_L1_loss'

In [7]:
from sklearn.externals import joblib

for trained_model in trained_models:
    model_dir = os.path.join(training_dir, trained_model)
    training_log = pd.read_csv(os.path.join(model_dir, 'training.csv'))
    min_index = training_log[[optimal_epoch_loss]].idxmin()
    min_epoch, min_loss = training_log.loc[min_index][['epoch', optimal_epoch_loss]].values[0]
    print("Model '%s', optimal loss: %.3f at epoch %d" % (trained_model, min_loss, min_epoch))
    
    epoch_weights_name = os.path.join(model_dir, 'weights.%04d.h5' % min_epoch)
    print(epoch_weights_name)
    model.load_weights(epoch_weights_name)
    eval_result = validation(model, dump_name = trained_model)
    joblib.dump(eval_result, 'metrics-raw-%s.dump' % trained_model)

Model 'weights', optimal loss: 78.969 at epoch 36
./training/weights/weights.0036.h5
loading annotations into memory...


  0%|          | 0/5000 [00:00<?, ?it/s]

Done (t=0.27s)
creating index...
index created!


100%|██████████| 5000/5000 [1:45:30<00:00,  1.24s/it]


Loading and preparing results...
DONE (t=0.40s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=3.18s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.553
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.769
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.594
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.526
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.606
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.590
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.783
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.626
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.535
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDe

In [8]:
weights_path = "model/keras/model.h5" # orginal weights converted from caffe
model.load_weights(weights_path)
eval_result_original = validation(model, dump_name = 'original')
joblib.dump(eval_result_original, 'metrics-raw-original.dump')

loading annotations into memory...


  0%|          | 0/5000 [00:00<?, ?it/s]

Done (t=0.40s)
creating index...
index created!


100%|██████████| 5000/5000 [1:41:50<00:00,  1.25s/it]


Loading and preparing results...
DONE (t=0.21s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=3.54s).
Accumulating evaluation results...
DONE (t=0.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.578
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.789
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.632
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.559
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.613
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.616
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.808
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.662
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.569
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDe

['metrics-raw-original.dump']

In [9]:
raw_eval_list = !ls metrics-raw*.dump

In [10]:
from sklearn.externals import joblib

for raw_eval in raw_eval_list:
    eval_result = joblib.load(raw_eval)
    print("\n" + raw_eval)
    eval_result.summarize()
    scores = per_image_scores(eval_result)
    scores.to_csv('%s-scores.csv' % raw_eval)
    print("Average per-image score (not coco metric): %.3f" % scores['average'].mean())


metrics-raw-original.dump
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.578
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.789
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.632
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.559
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.613
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.616
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.808
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.662
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.569
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.686


/opt/home/anatolix/iidf/cv/keras_Realtime_Multi-Person_Pose_Estimation/coco_metric.py:344: RuntimeWarning: invalid value encountered in double_scalars
  scores['average'] = np.mean(np.sum(matches != 0, axis=1)) / scores['gt_person_count']


Average per-image score (not coco metric): 0.639

metrics-raw-weights-cpp-lr.dump
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.521
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.730
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.564
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.495
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.566
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.556
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.748
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.505
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.631
Average per-image score (not coco metric): 0.598

metrics-raw-weights.dump
 Average Precision  (AP) @[ IoU=0.50:0.95 |